# This is a scratch notebook for development
The final entry function is get_details_from_llm in restaurant_details_from_llm.py

## Import Libraries

In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv, find_dotenv
from langchain import PromptTemplate
from langchain.output_parsers import PydanticOutputParser
from pydantic import Field, BaseModel

import os

## LLM Model and API Key

In [3]:
# Locate .env file, and load environment variables.
dotenv_path = find_dotenv()
if dotenv_path:
    load_dotenv(dotenv_path)
else:
    raise FileNotFoundError(".env file not found!")


# Set the model name for our LLMs.
GEMINI_MODEL = "gemini-2.0-flash"

# Load the API key to a variable.
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
if not GEMINI_API_KEY:
    raise ValueError("GEMINI_API_KEY is missing! Check your .env file.")


## Create the LangChain Model

In [4]:
# Initialize the model.
llm = ChatGoogleGenerativeAI(model=GEMINI_MODEL, google_api_key=GEMINI_API_KEY, temperature=0.3)

In [5]:
# restaurant_name = "Pho Binh"
# restaurant_city = "Houston"
# restaurant_street = "Beechnut"

restaurant_name = "Biryani Pot"
restaurant_city = "Houston"
restaurant_street = "Westheimer"

# restaurant_name = "Chama Gaucha Brazilian Steakhouse"
# restaurant_city = "Houston"
# restaurant_street = "Westheimer"

# Define a query as a string.
# query = f"Give me a summary and review of {restaurant_name} in {restaurant_city} on {restaurant_street}"

query = f"Give me the details of {restaurant_name} in {restaurant_city} on {restaurant_street}"

# Pass the query to the invoke method and print the result.
# response = llm.invoke(query)
# print(response.content)

## Templates for Instructions

In [6]:
# Define the format for the template.
format = """You are a world famous restaurant expert.  Answer only questions that would be about restaurants.
If the human asks questions not related to restaurant, remind them that your job is to help them get the details of a restaurnt
Question: {query}
Answer:

    **Your response should have the following information:**

    **Summary:** [Summary of the restaurant]
    **Phone:** [Phone number of the restaurant]
    **Address:** [Address of the restaurent]
    **Moods:** [Moods of the restaurant]
    **Highlight:** [quality of food, service, value]
    **Rating:** [Rating]
    **Hours:** [hours of business]
    **Price:** [usual price range per person]
    **Popular Items:** [popluar menu items]

    For example:
    **Summary:** Chama Gaucha is a high-end churrascaria (Brazilian steakhouse) offering a wide selection of grilled meats carved tableside by gauchos (Brazilian cowboys)
    **Phone:** (713) 357-9777
    **Address:** 5655 Westheimer Rd Suite A, Houston, TX 77056
    **Moods:** Vibrant, lively, and celebratory
    **Highlight:** Reviewers consistently praise the "perfectly cooked meats" and the "robust salad bar" with fresh options. 
                The service is frequently described as "impeccable" and "attentive," with staff ensuring glasses are filled and meat preferences are met.
    **Rating:** 4.7
    **Hours:**  Sunday - Thursday: 11 AM - 10 PM
                Friday - Saturday: 11 AM - 11 PM
    **Price:**  $30 - $50
    **Popular Items:** Picanha, Churrasco, Salad Bar

    Do not include any extra text or formatting. 
"""

# Construct the prompt template.
prompt_template = PromptTemplate(
    input_variables=["query"], 
    template = format)

## Define the output parser and parsing instructions

In [7]:
# Define the class for our parsed responses.
class restaurant_detail(BaseModel):
    phone: str = Field(description="Phone")
    address: str = Field(description="Address")
    summary: str = Field(description="Summary")
    moods: str = Field(description="Moods")
    highlight: str = Field(description="Highlight")
    rating: str = Field(description="Rating")
    hours: str = Field(description="Hours")
    price: str = Field(description="Price")
    popular_items: str = Field(description="Popular Items")

# Initialize the output parser using the schema.
parser = PydanticOutputParser(pydantic_object = restaurant_detail)

# Get the output format instructions and print them.
instructions = parser.get_format_instructions()


In [8]:
# Combine the query with the instructions.
query += "\n\n" + instructions

# Pass the query to the invoke method, and print the result.
response = (prompt_template | llm).invoke(query)

# Parse the result, store it, and print it.
data = parser.parse(response.content)


In [9]:
# Display the data
print(data.phone + '\n')
print(data.address + '\n')
print(data.summary + '\n')
print(data.moods + '\n')
print(data.highlight + '\n')
print(data.rating + '\n')
print(data.hours + '\n')
print(data.price + '\n')
print(data.popular_items + '\n')


(832) 767-2525

650 Westheimer Rd Suite C, Houston, TX 77006

Biryani Pot is a restaurant that specializes in Indian and Pakistani cuisine, with a focus on biryani dishes.

Casual, family-friendly

Known for its flavorful biryanis and affordable prices. Many reviewers praise the generous portion sizes and authentic taste. Good value for money.

4.0

Sunday-Thursday 11 AM - 10 PM, Friday-Saturday 11 AM - 11 PM

$10 - $20

Chicken Biryani, Goat Biryani, Vegetable Biryani

